## Lab 3: DDI 

### Imports

In [4]:
import os
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.parse.corenlp import CoreNLPDependencyParser
from nltk.corpus import stopwords 
import xml.etree.ElementTree as ET


### Variables

In [10]:
inputdir = "data\Devel"
test_path = 'data/Test-DDI'
train_path = 'data/Train'
outputfile = 'task9.2_develGoal_1.txt'

my_parser = CoreNLPDependencyParser(url="http://localhost:9000")

### Functions

In [20]:
def analyze(sent):
    #mytree, = my_parser.raw_parse(sent)
    
    return

def check_interaction(analysis, entities,e1,e2):
    interaction = -1
    # Rules
    
    return interaction

def evaluate(inputdir, outputfile):
    os.system("java -jar eval/evaluateDDI.jar "+ str(inputdir) + " " + str(outputfile))
    return

def main_function(inputdir):
    # process each file in directory
    for filename in os.listdir(inputdir):
        # parse XML file, obtaining a DOM tree
        fullname = os.path.join(inputdir, filename)
        tree = ET.parse(fullname)
        root = tree.getroot()  
        
        # process each sentence in the file
        for sentence in root.findall('sentence'):
            # Get sentence id and tokenize text
            sent_id = sentence.get('id') # get sentence id
            sent_text = sentence.get('text') #get sentence text 
            print(sentence)
            print(sent_id)
            print(sent_text)
            # load sentence entities into a dictionary
            entities = {}
            for ent in root.findall('entity'):
                ent_id = ent.get('id') 
                offs = ent.get('charOffset').value.split('-')
                entities[ent_id] = offs
            # Tokenize, tag and parse sentence
            analysis = analyze(sent_text)
            
            # for each pair in the sentence, decide whether it is DDI and its type
            for pair in root.findall('pair'):
                id_e1 = pair.get('e1').value
                id_e2 = pair.get('e2').value
                (is_ddi, ddi_type) = check_interaction(analysis, entities, id_e1, id_e2)
    # get performance score
    evaluate(inputdir, outputfile)

In [21]:
main_function(inputdir)

<Element 'sentence' at 0x000002525A22D868>
DDI-MedLine.d78.s0
The fluoroquinolones for urinary tract infections: a review.

analyse
<Element 'sentence' at 0x000002525A22D7C8>
DDI-MedLine.d78.s1
The fluoroquinolones are a rapidly growing class of antibiotics with a broad spectrum of activity against gram-negative and some gram-positive aerobic bacteria. 
analyse
<Element 'sentence' at 0x000002525A22D688>
DDI-MedLine.d78.s2
These agents, including norfloxacin, ciprofloxacin, ofloxacin, enoxacin, and lomefloxacin, have been extensively studied and have demonstrated efficacy and safety profiles comparable to those of other traditional agents for the treatment of complicated or uncomplicated urinary tract infections and prostatitis. 
analyse
<Element 'sentence' at 0x000002525A22D098>
DDI-MedLine.d78.s3
Advantages offered by this class of antibiotics include optimal pharmacokinetics, effectiveness against multidrug-resistant organisms, and oral administration even when parenteral antibiotics

<Element 'sentence' at 0x000002525A22D6D8>
DDI-DrugBank.d46.s1
- Alcohol, barbiturates, or narcotics: Potentiation of otthostatic hypotension may occur
analyse
<Element 'sentence' at 0x000002525A22D048>
DDI-DrugBank.d46.s2
.
analyse
<Element 'sentence' at 0x000002525A22D598>
DDI-DrugBank.d46.s3
- Antidiabetic drugs: (Oral agents and insulin) Dosage adjustment of the antidiabetic drug may be required
analyse
<Element 'sentence' at 0x000002525A23C778>
DDI-DrugBank.d46.s4
.
analyse
<Element 'sentence' at 0x000002525A23C408>
DDI-DrugBank.d46.s5
- Other antihypertensive drugs: Additive effect or potentiation
analyse
<Element 'sentence' at 0x000002525A23C458>
DDI-DrugBank.d46.s6
.
analyse
<Element 'sentence' at 0x000002525A23CE08>
DDI-DrugBank.d46.s7
- Cholestyramine and colestipol resins: Cholestytamine and colestipol resins have the potential of binding thiazide diuretics and reducing diuretic absorption from the gastrointestinal tract
analyse
<Element 'sentence' at 0x000002525A231F48>
DDI

8.
analyse
<Element 'sentence' at 0x000002525A202DB8>
DDI-DrugBank.d499.s24
Lithium: Nonsteroidal anti-inflammatory agents have been reported to increase steadystate plasma lithium levels.
analyse
<Element 'sentence' at 0x000002525A28E778>
DDI-DrugBank.d499.s25
It is recommended that plasma lithium levels be monitored when ketoprofen is coadministered with lithium.
analyse
<Element 'sentence' at 0x000002525A28EF98>
DDI-DrugBank.d499.s26
DRUG/LABORATORY TEST INTERACTIONS: EFFECT ON BLOOD COAGULATION Ketoprofen decreases platelet adhesion and aggregation.
analyse
<Element 'sentence' at 0x000002525A28EAE8>
DDI-DrugBank.d499.s27
Therefore, it can prolong bleeding time by approximately 3 to 4 minutes from baseline values.
analyse
<Element 'sentence' at 0x000002525A28E4F8>
DDI-DrugBank.d499.s28
There is no significant change in platelet count, prothrombin time, partial thromboplastin time, or thrombin time.
analyse
<Element 'sentence' at 0x000002525A23D958>
DDI-DrugBank.d66.s0
Human growth h